In [23]:
from ultralytics import YOLO
import cv2 as cv
import time


# 1. 커스텀 학습된 YOLO 모델 가중치 파일 경로 (.pt)
MODEL_PATH = "wsl_seg_best.pt"

# 2. 라즈베리 파이 카메라 서버의 영상 스트림 주소 (URL)
VIDEO_STREAM_URL = 'http://192.168.0.124:8000/stream.mjpg'

# 3. 탐지 임계값 (Confidence Threshold)
CONF_THRESHOLD = 0.5 # 필요에 따라 조정하세요 (예: 0.3, 0.7 등)

# 4. IOU 임계값 (NMS IoU Threshold)
IOU_THRESHOLD = 0.5 # 필요에 따라 조정하세요 (예: 0.4, 0.6 등)

# 5. 결과 시각화 창 표시 여부
SHOW_DETECTION_WINDOW = True

# model= YOLO("wsl_best.pt")

# --- YOLO 모델 로드 ---
try:
    print(f"모델 로드 중: {MODEL_PATH}")
    model = YOLO(MODEL_PATH)
    print("모델 로드 완료.")
except Exception as e:
    print(f"모델 로드 오류가 발생했습니다: {e}")
    print("모델 파일 경로를 확인하거나, 파일이 손상되지 않았는지 확인하세요.")
    exit()

# --- 영상 스트림 열기 ---
print(f"영상 스트림 연결 중: {VIDEO_STREAM_URL}")
# cv2.VideoCapture 객체를 생성하여 영상 스트림을 엽니다.
# OpenCV가 FFmpeg와 함께 빌드되어 있어야 네트워크 스트림 처리가 가능합니다.
cap = cv.VideoCapture(VIDEO_STREAM_URL)

# 스트림이 제대로 열렸는지 확인
if not cap.isOpened():
    print(f"오류: 영상 스트림을 열 수 없습니다. 다음을 확인하세요:")
    print(f"- 영상 스트림 URL: {VIDEO_STREAM_URL} 이 올바른지.")
    print("- 라즈베리 파이 카메라 서버가 실행 중인지.")
    print("- 네트워크 방화벽 설정 (포트가 열려 있는지).")
    print("- OpenCV가 FFmpeg를 지원하며 네트워크 스트림 처리가 가능한지.")
    exit()
print("영상 스트림 연결 성공.")

# --- 영상 프레임 읽고 감지 수행 ---
print("\n영상 스트림으로부터 프레임을 읽고 감지를 시작합니다.")
print(f"감지 결과를 보려면 {SHOW_DETECTION_WINDOW} 설정을 확인하세요.")
print("'q' 키를 누르면 감지를 종료합니다.")

# Optional: FPS 계산을 위한 변수 초기화
# start_time = time.time()
# frame_count = 0

while True:
    ret, frame = cap.read()

    if not ret:
        print("영상 스트림에서 프레임을 더 이상 읽을 수 없습니다. 스트림이 종료되었거나 오류가 발생했습니다.")
        break

    # Optional: 읽은 프레임 크기 확인 (디버깅용)
    # if frame is not None:
    #     print(f"읽은 프레임 크기: {frame.shape}")

    # 현재 프레임에 대해 객체 감지 수행
    results = model.predict(
        source=frame,
        conf=CONF_THRESHOLD,
        # iou=IOU_THRESHOLD, # 필요하다면 IOU 임계값 설정
        verbose=False
    )

    # --- 불량 판별 로직 시작 ---

    # 이미지 전체 면적 계산 (높이 * 너비)
    # frame.shape는 (높이, 너비, 채널) 형태입니다.
    if frame is not None:
        image_height, image_width = frame.shape[:2]
        total_image_area = image_height * image_width
        print(f"이미지 크기: {image_width}x{image_height}, 전체 면적: {total_image_area}")

        # 'unriped' 불량 판별을 위한 면적 임계값 설정 (전체 이미지 면적의 백분율)
        # 이 값은 실제 적용 환경에 맞게 조정해야 합니다.
        UNRIPED_AREA_THRESHOLD_PERCENT = 5.0 # 예: 이미지 면적의 5% 이상일 때 불량 판정
        unriped_area_threshold_pixels = total_image_area * (UNRIPED_AREA_THRESHOLD_PERCENT / 100.0)
        print(f"'unriped' 불량 판정 면적 임계값: {UNRIPED_AREA_THRESHOLD_PERCENT:.2f}% ({unriped_area_threshold_pixels:.2f} 픽셀)")

        detected_defects = [] # 감지된 불량 정보를 저장할 리스트

        # 감지된 각 객체에 대해 반복
        # results[0]는 현재 프레임에 대한 감지 결과 객체입니다.
        if results and results[0].boxes: # 감지된 객체가 있는지 확인
            for i in range(len(results[0].boxes)):
                box = results[0].boxes[i]
                mask = results[0].masks[i] # 세그멘테이션 마스크 정보
                class_id = int(box.cls) # 클래스 인덱스
                confidence = float(box.conf) # 신뢰도 점수

                # 클래스 인덱스를 이름으로 변환
                class_name = model.names[class_id] if class_id in model.names else f"Unknown Class {class_id}"

                is_defect = False
                defect_reason = ""
                unriped_mask_area = 0 # 'unriped' 마스크 면적 저장 변수

                # 'Bruise' 클래스 판별
                if class_name == 'Bruise':
                    is_defect = True
                    defect_reason = "Bruise (항상 불량)"
                    print(f"불량 감지: {class_name} (신뢰도: {confidence:.2f}) - {defect_reason}")
                    detected_defects.append({'class': class_name, 'confidence': confidence, 'reason': defect_reason, 'box': box.xyxy[0].tolist()})

                # 'unriped' 클래스 판별 (면적 기준)
                elif class_name == 'unriped':
                    # 세그멘테이션 마스크의 면적 계산
                    # mask.data는 마스크 데이터 (텐서 형태)
                    # .sum()을 통해 마스크 픽셀의 합 (면적) 계산
                    # mask.data는 [N, H, W] 형태일 수 있으므로, 해당 객체의 마스크만 선택
                    if mask is not None and mask.data is not None:
                        # 마스크 데이터는 배치 차원(N)이 있을 수 있으므로 첫 번째(0번) 객체의 마스크를 사용
                        # 실제로 각 객체에 대해 반복하므로 mask 변수는 해당 객체의 마스크 데이터만 가지고 있을 것입니다.
                        # mask.data.sum() 또는 mask.xy[0].shape 등으로 면적 추정 가능
                        # 가장 정확한 방법은 mask.masks.data[i].sum() 또는 mask.data[0].sum() (텐서 형태일 경우)
                        # 또는 mask.xy 리스트의 폴리곤 좌표를 사용하여 면적 계산
                        # 여기서는 간단하게 mask.data가 제공하는 이진 마스크 텐서의 합을 사용합니다.
                        # Ultralytics 버전 및 설정에 따라 mask 데이터 형태가 다를 수 있습니다.
                        # mask.data는 [1, H, W] 형태의 불리언 텐서일 가능성이 높습니다.
                        try:
                             # mask.data는 [N, H, W] 또는 [H, W] 형태의 텐서일 수 있습니다.
                             # 해당 객체의 마스크 데이터만 가져와 합산합니다.
                             # results[0].masks.data[i] 또는 mask.data[0] 등 접근 방식 확인 필요
                             # 가장 일반적인 형태는 mask.data[0] (텐서) 또는 mask.masks.data[i] (텐서)
                             # 여기서는 mask.data가 해당 객체의 마스크 텐서라고 가정합니다.
                             if mask.data.ndim == 3: # 만약 [N, H, W] 형태라면 첫 번째 차원 선택
                                 unriped_mask_area = mask.data[0].sum().item()
                             else: # 만약 [H, W] 형태라면 그대로 합산
                                 unriped_mask_area = mask.data.sum().item()

                             unriped_area_percentage = (unriped_mask_area / total_image_area) * 100 if total_image_area > 0 else 0

                             print(f"감지: {class_name} (신뢰도: {confidence:.2f}), 면적: {unriped_mask_area:.2f} 픽셀 ({unriped_area_percentage:.2f}%)")

                             if unriped_area_percentage >= UNRIPED_AREA_THRESHOLD_PERCENT:
                                 is_defect = True
                                 defect_reason = f"unriped (area {unriped_area_percentage:.2f}% >= Threshold {UNRIPED_AREA_THRESHOLD_PERCENT:.2f}%)"
                                 print(f"  -> 불량 판정: {defect_reason}")
                                 detected_defects.append({'class': class_name, 'confidence': confidence, 'reason': defect_reason, 'box': box.xyxy[0].tolist(), 'area_percent': unriped_area_percentage})
                             else:
                                 # 불량 임계값 미만이므로 불량이 아님
                                 print(f"  -> 정상 판정: 면적 임계값 미만")

                        except Exception as e:
                            print(f"Error calculating mask area for {class_name}: {e}")
                            # 마스크 면적 계산 오류 시 해당 객체는 불량으로 판정하지 않거나, 필요에 따라 다른 처리
                            pass # 오류 발생 시 해당 객체는 불량으로 처리하지 않고 넘어감

                # 다른 불량 종류 ('Black Dot', 'scratch')에 대한 판별 로직은 여기에 추가
                # 예: 'Black Dot'이나 'scratch'는 항상 불량으로 판정하고 싶다면:
                elif class_name in ['Black Dot', 'scratch']:
                     is_defect = True
                     defect_reason = f"{class_name} (Always Defect)"
                     print(f"불량 감지: {class_name} (신뢰도: {confidence:.2f}) - {defect_reason}")
                     detected_defects.append({'class': class_name, 'confidence': confidence, 'reason': defect_reason, 'box': box.xyxy[0].tolist()})


            # 현재 프레임에서 감지된 최종 불량 목록 출력 또는 활용
            if detected_defects:
                print(f"\n--- 현재 프레임 불량 요약 ({len(detected_defects)}건) ---")
                for defect in detected_defects:
                    print(f"- 클래스: {defect['class']}, 사유: {defect['reason']}, 신뢰도: {defect['confidence']:.2f}")
                print("-------------------------------------\n")
            # else:
                # print("현재 프레임에서 불량 감지되지 않음.") # 너무 자주 출력될 수 있으므로 필요시 주석 해제

    # --- 불량 판별 로직 종료 ---


    # 감지 결과 프레임 가져오기 (불량 판별 결과와 시각화는 분리)
    # 필요에 따라 plot() 메소드 인자를 조절하여 시각화 방식을 변경할 수 있습니다.
    # 예: masks=False 로 설정하여 마스크를 그리지 않음
    annotated_frame = results[0].plot(masks=True) # 기본 시각화 (마스크 포함)

    # Optional: 불량 판별 결과에 따라 시각화에 추가 정보 그리기
    # 예를 들어, 불량으로 판정된 객체에만 특정 색상의 테두리를 그리거나 텍스트 추가
    if detected_defects and annotated_frame is not None:
         for defect in detected_defects:
             # defect['box']는 [x1, y1, x2, y2] 리스트 형태입니다.
             x1, y1, x2, y2 = map(int, defect['box'])
             label_text = f"Defect: {defect['class']}, Reason: {defect['reason']}"
             # OpenCV를 사용하여 annotated_frame에 직접 그리기 예시
             cv.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2) # 빨간색 박스
             cv.putText(annotated_frame, label_text, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


    # 결과 화면 표시 (설정된 경우)
    if SHOW_DETECTION_WINDOW and annotated_frame is not None:
        cv.imshow("YOLO Object Detection from RPi Stream", annotated_frame)

    # 키 입력을 대기하고 'q' 키가 눌리면 루프 종료
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# --- 자원 해제 ---
cap.release()
if SHOW_DETECTION_WINDOW:
    cv.destroyAllWindows()

print("\n감지 및 영상 스트림 처리가 종료되었습니다.")


모델 로드 중: wsl_seg_best.pt
모델 로드 완료.
영상 스트림 연결 중: http://192.168.0.124:8000/stream.mjpg
영상 스트림 연결 성공.

영상 스트림으로부터 프레임을 읽고 감지를 시작합니다.
감지 결과를 보려면 True 설정을 확인하세요.
'q' 키를 누르면 감지를 종료합니다.
이미지 크기: 640x480, 전체 면적: 307200
'unriped' 불량 판정 면적 임계값: 5.00% (15360.00 픽셀)
불량 감지: Bruise (신뢰도: 0.92) - Bruise (항상 불량)

--- 현재 프레임 불량 요약 (1건) ---
- 클래스: Bruise, 사유: Bruise (항상 불량), 신뢰도: 0.92
-------------------------------------

이미지 크기: 640x480, 전체 면적: 307200
'unriped' 불량 판정 면적 임계값: 5.00% (15360.00 픽셀)
불량 감지: Bruise (신뢰도: 0.88) - Bruise (항상 불량)

--- 현재 프레임 불량 요약 (1건) ---
- 클래스: Bruise, 사유: Bruise (항상 불량), 신뢰도: 0.88
-------------------------------------

이미지 크기: 640x480, 전체 면적: 307200
'unriped' 불량 판정 면적 임계값: 5.00% (15360.00 픽셀)
불량 감지: Bruise (신뢰도: 0.92) - Bruise (항상 불량)

--- 현재 프레임 불량 요약 (1건) ---
- 클래스: Bruise, 사유: Bruise (항상 불량), 신뢰도: 0.92
-------------------------------------

이미지 크기: 640x480, 전체 면적: 307200
'unriped' 불량 판정 면적 임계값: 5.00% (15360.00 픽셀)
불량 감지: Bruise (신뢰도: 0.93) - Bruise (항상 불량)

--- 현재 프레임 불량 

In [ ]:
# 필요한 라이브러리 임포트
from ultralytics import YOLO
import cv2 as cv
import time
import numpy as np
from PIL import ImageFont, ImageDraw, Image # Pillow 모듈 임포트
import torch # Ultralytics 결과(텐서) 처리에 필요
import paho.mqtt.client as mqtt # MQTT 클라이언트 라이브러리 임포트
import json # 불량 정보를 JSON 형태로 보내기 위해 임포트

# --- 설정 변수 ---

# 1. 커스텀 학습된 YOLO 모델 가중치 파일 경로 (.pt)
MODEL_PATH = "wsl_seg_best.pt"

# 2. 라즈베리 파이 카메라 서버의 영상 스트림 주소 (URL)
VIDEO_STREAM_URL = 'http://192.168.0.124:8000/stream.mjpg'

# 3. 탐지 임계값 (Confidence Threshold) - 모델이 객체를 얼마나 확신할 때 감지할지 결정
CONF_THRESHOLD = 0.5 # 필요에 따라 조정하세요 (예: 0.3, 0.7 등)

# 4. IOU 임계값 (NMS IoU Threshold) - 겹치는 바운딩 박스 중 하나만 남길 기준
IOU_THRESHOLD = 0.5 # 필요에 따라 조정하세요 (예: 0.4, 0.6 등)

# 5. 결과 시각화 창 표시 여부
SHOW_DETECTION_WINDOW = True

# 6. 한글 폰트 파일 경로 및 크기 설정 (Pillow 사용)
# 시스템에 설치된 한글 폰트 파일 경로를 사용하세요.
# 예: Windows의 경우 C:/Windows/Fonts/gulim.ttf
# 예: Linux의 경우 /usr/share/fonts/truetype/nanum/NanumGothic.ttf 등
# 폰트 파일이 없다면 다운로드하여 프로젝트 폴더에 넣고 해당 경로를 지정하세요.
FONT_PATH = "DetectionModule/src/PretendardVariable.ttf" # 실제 폰트 파일 경로로 변경하세요.
FONT_SIZE = 20 # 폰트 크기 (픽셀 단위)

# 7. 'unriped' 불량 판별을 위한 사과 면적 대비 임계값 설정 (사과 면적의 백분율)
# 사과 객체 내에서 'unriped' 영역이 이 비율 이상을 차지할 때 불량으로 판정합니다.
UNRIPED_PERCENT_OF_APPLE_THRESHOLD = 10.0 # 예: 사과 면적의 10% 이상일 때 불량 판정

# --- MQTT 설정 변수 ---
MQTT_BROKER_HOST = "192.168.0.124" # MQTT 브로커 주소 (라즈베리 파이 또는 다른 서버 주소)
MQTT_BROKER_PORT = 1883        # MQTT 브로커 포트 (일반적으로 1883)
MQTT_TOPIC_DEFECT = "defect_detection/status" # 불량 감지 상태를 보낼 토픽
MQTT_TOPIC_DETAILS = "defect_detection/details" # 불량 상세 정보를 보낼 토픽

# --- 함수 정의 ---

# YOLO 모델 로드 함수
def load_yolo_model(model_path):
    """
    지정된 경로에서 YOLO 모델 가중치 파일을 로드합니다.

    Args:
        model_path (str): YOLO 모델 가중치 파일 (.pt) 경로.

    Returns:
        YOLO: 로드된 YOLO 모델 객체, 로드 실패 시 None 반환.
    """
    print(f"모델 로드 중: {model_path}")
    try:
        model = YOLO(model_path)
        print("모델 로드 완료.")
        return model
    except Exception as e:
        print(f"모델 로드 오류가 발생했습니다: {e}")
        print("모델 파일 경로를 확인하거나, 파일이 손상되지 않았는지 확인하세요.")
        return None

# 영상 스트림 열기 함수
def open_video_stream(stream_url):
    """
    지정된 URL의 영상 스트림을 엽니다.

    Args:
        stream_url (str): 영상 스트림 URL (예: 'http://.../stream.mjpg').

    Returns:
        cv2.VideoCapture: 열린 VideoCapture 객체, 열기 실패 시 None 반환.
    """
    print(f"영상 스트림 연결 중: {stream_url}")
    cap = cv.VideoCapture(stream_url)

    if not cap.isOpened():
        print(f"오류: 영상 스트림을 열 수 없습니다. 다음을 확인하세요:")
        print(f"- 영상 스트림 URL: {stream_url} 이 올바른지.")
        print("- 라즈베리 파이 카메라 서버가 실행 중인지.")
        print("- 네트워크 방화벽 설정 (포트가 열려 있는지).")
        print("- OpenCV가 FFmpeg를 지원하며 네트워크 스트림 처리가 가능한지.")
        return None
    print("영상 스트림 연결 성공.")
    return cap

# 한글 텍스트를 이미지에 그리는 함수 (Pillow 사용)
def draw_korean_text(img, text, pos, font_path, font_size, text_color=(255, 255, 255)):
    """
    OpenCV 이미지에 한글 텍스트를 그립니다.

    Args:
        img (numpy.ndarray): 텍스트를 그릴 OpenCV 이미지 (BGR).
        text (str): 그릴 텍스트 문자열.
        pos (tuple): 텍스트의 좌측 상단 좌표 (x, y).
        font_path (str): 사용할 한글 폰트 파일 경로.
        font_size (int): 폰트 크기.
        text_color (tuple): 텍스트 색상 (BGR 튜플).

    Returns:
        numpy.ndarray: 텍스트가 그려진 OpenCV 이미지 (BGR).
    """
    # OpenCV 이미지를 PIL 이미지로 변환 (BGR -> RGB)
    img_pil = Image.fromarray(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)

    try:
        # 유니코드 폰트 로드
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print(f"오류: 폰트 파일을 찾을 수 없습니다: {font_path}. 기본 폰트를 사용합니다.")
        font = ImageFont.load_default()
        text = "폰트 오류" # 오류 메시지 표시 (기본 폰트는 한글 깨짐)

    # 텍스트 그리기
    draw.text(pos, text, font=font, fill=text_color)

    # PIL 이미지를 다시 OpenCV 이미지(NumPy 배열)로 변환하여 반환 (RGB -> BGR)
    return cv.cvtColor(np.array(img_pil), cv.COLOR_RGB2BGR)

# 객체 감지 및 세그멘테이션 추론 수행 함수
def perform_inference(model, frame, conf_threshold, iou_threshold):
    """
    주어진 프레임에 대해 YOLO 모델 추론을 수행합니다.

    Args:
        model (YOLO): 로드된 YOLO 모델 객체.
        frame (numpy.ndarray): 추론을 수행할 이미지 프레임.
        conf_threshold (float): 신뢰도 임계값.
        iou_threshold (float): NMS IOU 임계값.

    Returns:
        Results: YOLO 추론 결과 객체.
    """
    results = model.predict(
        source=frame,
        conf=conf_threshold,
        iou=iou_threshold,
        verbose=False # 추론 과정 상세 출력 끔
    )
    return results

# 감지 결과를 분석하여 불량 판별 함수
def analyze_defects(results, model_names, unriped_threshold_percent):
    """
    YOLO 감지 결과를 분석하여 불량을 판별합니다.

    Args:
        results (Results): YOLO 추론 결과 객체.
        model_names (dict): 클래스 인덱스와 이름 매핑 딕셔너리.
        unriped_threshold_percent (float): 사과 면적 대비 덜 익음 영역의 백분율 임계값.

    Returns:
        list: 감지된 불량 정보를 담은 딕셔너리 리스트.
    """
    detected_defects = []

    # 감지 결과가 있는지 확인
    if results and results[0].boxes and results[0].masks:
        boxes = results[0].boxes # 바운딩 박스 결과
        masks = results[0].masks # 세그멘테이션 마스크 결과 (텐서 형태)
        names = model_names      # 클래스 이름 매핑

        # 클래스별 인덱스 찾기
        try:
            apple_class_id = list(names.keys())[list(names.values()).index('Apple')]
            unriped_class_id = list(names.keys())[list(names.values()).index('unriped')]
            bruise_class_id = list(names.keys())[list(names.values()).index('Bruise')]
            black_dot_class_id = list(names.keys())[list(names.values()).index('Black Dot')]
            scratch_class_id = list(names.keys())[list(names.values()).index('scratch')]
        except ValueError as e:
            print(f"오류: 모델 클래스 이름에 예상치 못한 값이 있습니다: {e}")
            print(f"모델 클래스 목록: {names}")
            # 필요한 클래스 이름이 없으면 불량 판별 로직을 건너뛸 수 있습니다.
            apple_class_id = unriped_class_id = bruise_class_id = black_dot_class_id = scratch_class_id = -1 # 유효하지 않은 ID로 설정


        # 'Bruise', 'Black Dot', 'scratch'는 항상 불량으로 판정
        for i in range(len(boxes)):
            class_id = int(boxes.cls[i])
            confidence = float(boxes.conf[i])
            class_name = names.get(class_id, f"Unknown Class {class_id}")
            box_coords = boxes.xyxy[i].tolist()

            if class_id == bruise_class_id:
                 defect_reason = "멍 (항상 불량)"
                 # print(f"불량 감지: {class_name} (신뢰도: {confidence:.2f}) - {defect_reason}") # 너무 자주 출력될 수 있으므로 주석 처리
                 detected_defects.append({'class': class_name, 'confidence': confidence, 'reason': defect_reason, 'box': box_coords})

            elif class_id == black_dot_class_id:
                 defect_reason = "검은 점 (항상 불량)"
                 # print(f"불량 감지: {class_name} (신뢰도: {confidence:.2f}) - {defect_reason}") # 너무 자주 출력될 수 있으므로 주석 처리
                 detected_defects.append({'class': class_name, 'confidence': confidence, 'reason': defect_reason, 'box': box_coords})

            elif class_id == scratch_class_id:
                 defect_reason = "긁힘 (항상 불량)"
                 # print(f"불량 감지: {class_name} (신뢰도: {confidence:.2f}) - {defect_reason}") # 너무 자주 출력될 수 있으므로 주석 처리
                 detected_defects.append({'class': class_name, 'confidence': confidence, 'reason': defect_reason, 'box': box_coords})


        # 'Apple' 객체와 'unriped' 객체 필터링
        apple_indices = (boxes.cls == apple_class_id).nonzero(as_tuple=True)[0]
        unriped_indices = (boxes.cls == unriped_class_id).nonzero(as_tuple=True)[0]

        # 'unriped' 면적에 따른 'Apple' 불량 판정 로직
        if len(apple_indices) > 0 and len(unriped_indices) > 0:
            apple_masks_tensor = masks.data[apple_indices] # [Num_Apples, H, W]
            unriped_masks_tensor = masks.data[unriped_indices] # [Num_Unriped, H, W]
            apple_boxes = boxes[apple_indices] # Apple 객체의 box 정보

            # 각 Apple 객체에 대해 반복
            for j in range(len(apple_indices)):
                current_apple_mask = apple_masks_tensor[j] # 현재 Apple의 마스크 [H, W]
                current_apple_box = apple_boxes[j]         # 현재 Apple의 box 정보
                current_apple_confidence = float(current_apple_box.conf) # 현재 Apple의 신뢰도
                current_apple_box_coords = current_apple_box.xyxy[0].tolist() # 현재 Apple의 박스 좌표

                apple_area = current_apple_mask.sum().item() # 현재 Apple 마스크 면적

                if apple_area > 0: # Apple 면적이 0보다 클 경우에만 계산
                    # 현재 Apple 마스크와 모든 unriped 마스크의 교집합 계산
                    # 결과는 [Num_Unriped, H, W] 형태의 텐서
                    # 각 unriped 마스크가 현재 Apple 마스크와 겹치는 영역을 나타냄
                    intersection_with_all_unriped = current_apple_mask * unriped_masks_tensor

                    # 모든 unriped 마스크가 현재 Apple과 겹치는 총 면적 계산
                    # [Num_Unriped, H, W] -> H, W 차원을 합산 -> [Num_Unriped]
                    # 그 후 Num_Unriped 차원을 합산하여 총 픽셀 수 얻기
                    total_unriped_area_on_this_apple = intersection_with_all_unriped.sum().item()

                    # 현재 Apple 면적 대비 unriped 영역 비율 계산
                    unriped_on_apple_percentage = (total_unriped_area_on_this_apple / apple_area) * 100

                    # print(f"Apple {j+1}: 면적 {apple_area:.2f} 픽셀, Unriped 겹침 총 면적 {total_unriped_area_on_this_apple:.2f} 픽셀, 비율 {unriped_on_apple_percentage:.2f}%") # 디버깅 출력

                    # 비율이 임계값 이상이면 불량 판정
                    if unriped_on_apple_percentage >= unriped_threshold_percent:
                        defect_reason = f"덜 익음 (사과 면적 대비 {unriped_on_apple_percentage:.2f}% >= 임계값 {unriped_threshold_percent:.2f}%)"
                        # print(f"불량 판정: Apple (신뢰도: {current_apple_confidence:.2f}) - {defect_reason}") # 너무 자주 출력될 수 있으므로 주석 처리
                        # 불량 리스트에 Apple 객체 정보와 불량 사유 추가
                        # 이미 다른 이유로 추가되지 않았다면 추가 (중복 방지)
                        is_already_added = any(d['box'] == current_apple_box_coords for d in detected_defects)
                        if not is_already_added:
                             detected_defects.append({
                                 'class': 'Apple',
                                 'confidence': current_apple_confidence,
                                 'reason': defect_reason,
                                 'box': current_apple_box_coords,
                                 'area_percent_on_apple': unriped_on_apple_percentage
                             })
                        else:
                             # 이미 추가된 경우, 사유에 덜 익음 정보 추가 (예: 멍 + 덜 익음)
                             for d in detected_defects:
                                 if d['box'] == current_apple_box_coords:
                                     d['reason'] += f", {defect_reason}"
                                     break

    # 현재 프레임에서 감지된 최종 불량 목록 출력 또는 활용
    if detected_defects:
        print(f"\n--- 현재 프레임 불량 요약 ({len(detected_defects)}건) ---")
        for defect in detected_defects:
            print(f"- 클래스: {defect['class']}, 사유: {defect['reason']}, 신뢰도: {defect['confidence']:.2f}")
        print("-------------------------------------\n")
    # else:
        # print("현재 프레임에서 불량 감지되지 않음.") # 너무 자주 출력될 수 있으므로 필요시 주석 해제

    return detected_defects

# 감지 및 불량 판별 결과를 이미지에 시각화 함수
def visualize_results(annotated_frame, detected_defects, font_path, font_size):
    """
    감지 및 불량 판별 결과를 이미지에 시각화합니다.

    Args:
        annotated_frame (numpy.ndarray): YOLO plot() 메소드로 기본 시각화된 이미지 프레임.
        detected_defects (list): 감지된 불량 정보를 담은 딕셔너리 리스트.
        font_path (str): 한글 폰트 파일 경로.
        font_size (int): 폰트 크기.

    Returns:
        numpy.ndarray: 불량 정보가 추가로 시각화된 이미지 프레임.
    """
    # 불량 판별 결과에 따라 시각화에 추가 정보 그리기 (한글 텍스트 포함)
    if detected_defects and annotated_frame is not None:
         for defect in detected_defects:
             # defect['box']는 [x1, y1, x2, y2] 리스트 형태입니다.
             x1, y1, x2, y2 = map(int, defect['box'])
             label_text = f"불량: {defect['reason']}"
             # OpenCV를 사용하여 annotated_frame에 직접 그리기 예시 (불량 객체 박스)
             # 불량으로 판정된 객체에 빨간색 박스를 그립니다.
             cv.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2) # 빨간색 박스

             # Pillow 함수를 사용하여 한글 텍스트 그리기
             # 텍스트 위치는 박스 좌측 상단 약간 위 (y1 - 폰트 크기)로 설정
             text_pos = (x1, y1 - font_size - 5) # 5픽셀 여백 추가
             # 이미지가 너무 작아 텍스트 위치가 벗어나지 않도록 조정
             if text_pos[1] < 0:
                 text_pos = (x1, y1 + 5) # 박스 바로 아래에 그리기

             # 이미지 경계를 벗어나지 않도록 텍스트 위치 최종 조정
             text_pos = (max(0, text_pos[0]), max(0, text_pos[1]))


             annotated_frame = draw_korean_text(
                 annotated_frame,
                 label_text,
                 text_pos,
                 font_path,
                 font_size,
                 text_color=(0, 0, 255) # 텍스트 색상 (빨간색)
             )

    return annotated_frame

# MQTT 클라이언트 연결 콜백 함수
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("MQTT 브로커 연결 성공")
    else:
        print(f"MQTT 브로커 연결 실패: {rc}")

# MQTT 클라이언트 초기화 및 연결 함수
def initialize_mqtt_client(broker_host, broker_port):
    """
    MQTT 클라이언트를 초기화하고 브로커에 연결합니다.

    Args:
        broker_host (str): MQTT 브로커 주소.
        broker_port (int): MQTT 브로커 포트.

    Returns:
        mqtt.Client: 초기화되고 연결된 MQTT 클라이언트 객체, 연결 실패 시 None 반환.
    """
    print(f"MQTT 브로커 연결 시도: {broker_host}:{broker_port}")
    client = mqtt.Client()
    client.on_connect = on_connect # 연결 콜백 함수 설정

    try:
        client.connect(broker_host, broker_port, 60)
        # 네트워크 루프 시작 (백그라운드 스레드에서 실행)
        client.loop_start()
        # 연결이 완료될 때까지 잠시 대기
        time.sleep(1) # 필요에 따라 조정
        return client
    except Exception as e:
        print(f"MQTT 브로커 연결 중 오류 발생: {e}")
        return None

# MQTT 메시지 발행 함수
def publish_mqtt_message(client, topic, message):
    """
    지정된 토픽으로 MQTT 메시지를 발행합니다.

    Args:
        client (mqtt.Client): 연결된 MQTT 클라이언트 객체.
        topic (str): 메시지를 발행할 토픽.
        message (str): 발행할 메시지 내용.
    """
    if client and client.is_connected():
        try:
            # 메시지를 바이트 형태로 인코딩하여 발행
            result = client.publish(topic, message.encode('utf-8'), qos=1) # QoS 레벨 1 사용
            # print(f"MQTT 메시지 발행 결과: {result}") # 디버깅용
        except Exception as e:
            print(f"MQTT 메시지 발행 중 오류 발생: {e}")
    # else:
        # print("MQTT 클라이언트가 연결되지 않았습니다. 메시지를 발행할 수 없습니다.") # 너무 자주 출력될 수 있으므로 주석 처리


# 메인 처리 루프 함수
def main_loop(model_path, stream_url, conf_threshold, iou_threshold, show_window, font_path, font_size, unriped_threshold_percent, mqtt_broker_host, mqtt_broker_port, mqtt_topic_defect, mqtt_topic_details):
    """
    영상 스트림에서 프레임을 읽고 감지, 판별, 시각화 과정을 반복합니다.
    불량 감지 시 MQTT 신호를 보냅니다.
    """
    # 1. 모델 로드
    model = load_yolo_model(model_path)
    if model is None:
        return # 모델 로드 실패 시 종료

    # 2. 영상 스트림 열기
    cap = open_video_stream(stream_url)
    if cap is None:
        return # 스트림 열기 실패 시 종료

    # 3. MQTT 클라이언트 초기화 및 연결
    mqtt_client = initialize_mqtt_client(mqtt_broker_host, mqtt_broker_port)
    # MQTT 연결 실패 시에도 프로그램은 계속 실행되도록 합니다. (필요에 따라 종료하도록 수정 가능)
    if mqtt_client is None:
        print("MQTT 연결에 실패했습니다. 불량 감지 시 MQTT 메시지를 보내지 않습니다.")


    print("\n영상 스트림으로부터 프레임을 읽고 감지를 시작합니다.")
    print(f"감지 결과를 보려면 {show_window} 설정을 확인하세요.")
    print("'q' 키를 누르면 감지를 종료합니다.")

    # Optional: FPS 계산을 위한 변수 초기화
    # start_time = time.time()
    # frame_count = 0

    while True:
        # 영상 스트림에서 프레임 하나를 읽어옵니다.
        ret, frame = cap.read()

        # 프레임을 제대로 읽지 못했다면 (예: 스트림 종료, 연결 끊김 등) 루프 종료
        if not ret:
            print("영상 스트림에서 프레임을 더 이상 읽을 수 없습니다. 스트림이 종료되었거나 오류가 발생했습니다.")
            break

        # 4. 객체 감지 및 세그멘테이션 추론 수행
        results = perform_inference(model, frame, conf_threshold, iou_threshold)

        # 5. 감지 결과를 분석하여 불량 판별
        detected_defects = analyze_defects(results, model.names, unriped_threshold_percent)

        # --- 불량 감지 시 MQTT 신호 보내기 ---
        if detected_defects:
            # 불량 감지 상태 신호 보내기 (간단한 메시지)
            status_message = "Defect Detected"
            publish_mqtt_message(mqtt_client, mqtt_topic_defect, status_message)
            print(f"MQTT 상태 메시지 발행: {status_message} to {mqtt_topic_defect}")

            # 불량 상세 정보 보내기 (JSON 형태)
            # 감지된 불량 목록을 JSON 문자열로 변환
            details_message = json.dumps(detected_defects, indent=4) # 보기 좋게 들여쓰기 추가
            publish_mqtt_message(mqtt_client, mqtt_topic_details, details_message)
            print(f"MQTT 상세 메시지 발행: {details_message} to {mqtt_topic_details}")
        # else:
            # 불량 감지되지 않았을 때 정상 상태 신호를 보내고 싶다면 여기에 추가
            # publish_mqtt_message(mqtt_client, mqtt_topic_defect, "No Defect")


        # 6. 감지 결과 기본 시각화 (Ultralytics plot 사용)
        # plot() 메소드로 마스크, 박스만 그리고 라벨/신뢰도는 제외하여 한글 깨짐 방지
        # results[0] 객체가 비어있을 수 있으므로 확인 후 plot 호출
        annotated_frame = frame.copy() # 기본 프레임 복사
        if results and results[0].masks is not None: # 마스크 결과가 있을 때만 plot 시도
             annotated_frame = results[0].plot(masks=True, boxes=True, labels=False, conf=False)
        elif results and results[0].boxes is not None: # 마스크는 없지만 박스 결과가 있을 때
             annotated_frame = results[0].plot(masks=False, boxes=True, labels=False, conf=False)


        # 7. 불량 판별 결과에 따라 시각화에 추가 정보 그리기 (한글 텍스트 포함)
        annotated_frame = visualize_results(annotated_frame, detected_defects, font_path, font_size)

        # 8. 결과 화면 표시 (설정된 경우)
        if show_window and annotated_frame is not None:
            cv.imshow("YOLO Object Detection from RPi Stream", annotated_frame)

        # Optional: FPS 계산 및 출력
        # frame_count += 1
        # if (time.time() - start_time) > 1: # 1초마다 FPS 계산
        #    fps = frame_count / (time.time() - start_time)
        #    print(f"처리 FPS: {fps:.2f}")
        #    frame_count = 0
        #    start_time = time.time()

        # 키 입력을 대기하고 'q' 키가 눌리면 루프 종료
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    # --- 자원 해제 ---
    cap.release() # cv2.VideoCapture 객체 해제 (스트림 연결 종료)
    if show_window:
        cv.destroyAllWindows() # 생성된 모든 OpenCV 창 닫기

    # MQTT 클라이언트 연결 종료
    if mqtt_client:
        mqtt_client.loop_stop() # 네트워크 루프 중지
        mqtt_client.disconnect() # 브로커 연결 해제
        print("MQTT 클라이언트 연결 종료.")

    print("\n감지 및 영상 스트림 처리가 종료되었습니다.")

# --- 스크립트 실행 ---
if __name__ == "__main__":
    main_loop(
        MODEL_PATH,
        VIDEO_STREAM_URL,
        CONF_THRESHOLD,
        IOU_THRESHOLD,
        SHOW_DETECTION_WINDOW,
        FONT_PATH,
        FONT_SIZE,
        UNRIPED_PERCENT_OF_APPLE_THRESHOLD,
        MQTT_BROKER_HOST, # MQTT 브로커 주소 전달
        MQTT_BROKER_PORT, # MQTT 브로커 포트 전달
        MQTT_TOPIC_DEFECT, # MQTT 상태 토픽 전달
        MQTT_TOPIC_DETAILS # MQTT 상세 정보 토픽 전달
    )


<>:33: SyntaxWarning: invalid escape sequence '\s'
<>:33: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_13948\2052371082.py:33: SyntaxWarning: invalid escape sequence '\s'
  FONT_PATH = "DetectionModule\src\PretendardVariable.ttf" # 실제 폰트 파일 경로로 변경하세요.


모델 로드 중: wsl_seg_best.pt
모델 로드 완료.
영상 스트림 연결 중: http://192.168.0.124:8000/stream.mjpg
영상 스트림 연결 성공.
MQTT 브로커 연결 시도: 192.168.0.124:1883


C:\Users\Admin\AppData\Local\Temp\ipykernel_13948\2052371082.py:341: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


MQTT 브로커 연결 중 오류 발생: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
MQTT 연결에 실패했습니다. 불량 감지 시 MQTT 메시지를 보내지 않습니다.

영상 스트림으로부터 프레임을 읽고 감지를 시작합니다.
감지 결과를 보려면 True 설정을 확인하세요.
'q' 키를 누르면 감지를 종료합니다.

--- 현재 프레임 불량 요약 (1건) ---
- 클래스: Bruise, 사유: 멍 (항상 불량), 신뢰도: 0.65
-------------------------------------

MQTT 상태 메시지 발행: Defect Detected to defect_detection/status
MQTT 상세 메시지 발행: [
    {
        "class": "Bruise",
        "confidence": 0.6492542028427124,
        "reason": "\uba4d (\ud56d\uc0c1 \ubd88\ub7c9)",
        "box": [
            488.2093811035156,
            107.35993957519531,
            535.2084350585938,
            149.59249877929688
        ]
    }
] to defect_detection/details

--- 현재 프레임 불량 요약 (1건) ---
- 클래스: Bruise, 사유: 멍 (항상 불량), 신뢰도: 0.74
-------------------------------------

MQTT 상태 메시지 발행: Defect Detected to defect_detection/status
MQTT 상세 메시지 발행: [
    {
        "class": "Bruise",
        "confidence": 0.7383531928062439,
        "reason": "\uba4d (\ud56d\uc0c1 \ubd88